In [1]:
import pennylane as qml
from pennylane import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import cv2
import random
import os
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import random
from PIL import Image

In [2]:
dataset = "cat_v1"
categories = ["Bengal", "Domestic_Shorthair", "Maine_Coon", "Ragdoll", "Siamese"]
num_images = 500

In [3]:
image_size = (256, 256) 

In [4]:
def nearest_power_of_2(n):
    return 2 ** int(np.ceil(np.log2(n)))

In [5]:
for category in categories:
    folder_path = os.path.join(dataset, category)
    images = os.listdir(folder_path)[:num_images] 

In [6]:
quantum_encoded_images = []

In [7]:
for category in categories:
    folder_path = os.path.join(dataset, category)
    image_paths = [os.path.join(folder_path, img) for img in images]
    images = os.listdir(folder_path)[:num_images] 

    for img_file in tqdm(images, desc=f"Encoding {category} images"):
        try:
            
            img_path = os.path.join(folder_path, img_file)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, image_size)  
            image = image.flatten() / 256.0 

            target_size = nearest_power_of_2(len(image)) 
            num_qubits = int(np.log2(target_size))  


            dev = qml.device("default.qubit", wires=num_qubits)

            @qml.qnode(dev)
            def quantum_image_encoding(image):
                qml.AmplitudeEmbedding(features=image, wires=range(num_qubits), normalize=True, pad_with=0.0)
                return qml.state()

            padded_image = np.pad(image, (0, target_size - len(image)), 'constant')

            quantum_state = quantum_image_encoding(padded_image)
            quantum_encoded_images.append((category, img_file, quantum_state))
        
        except Exception as e:
            print(f"Error processing {img_file}: {e}")

Encoding Siamese images: 100%|██████████| 208/208 [00:11<00:00, 17.68it/s]


In [8]:
with open("quantum_encoded_pet_images.pkl", "wb") as f:
    pickle.dump(quantum_encoded_images, f)

In [9]:
print("Encoding Sucessfull!")

Encoding Sucessfull!


In [10]:
with open("quantum_encoded_pet_images.pkl", "rb") as f:
    loaded_data = pickle.load(f)

In [13]:
random_entries = random.sample(loaded_data, 10)
for i, (category, filename, quantum_state) in enumerate(random_entries):
    print(f"\n Entry {i+1}")
    print(f"Category : {category}")
    print(f"Filename : {filename}")
    print(f" Quantum State (Size: {len(quantum_state)}):")
    print(quantum_state[:10])


 Entry 1
Category : Domestic_Shorthair
Filename : american-shorthair-cat-856159.jpg
 Quantum State (Size: 65536):
[0.0056222 +0.j 0.00610325+0.j 0.00553201+0.j 0.00484051+0.j
 0.00402875+0.j 0.00411894+0.j 0.00478038+0.j 0.0036379 +0.j
 0.00369803+0.j 0.00481044+0.j]

 Entry 2
Category : Bengal
Filename : fullsizeoutput_63f0.jpg
 Quantum State (Size: 65536):
[0.00471666+0.j 0.00469606+0.j 0.00477845+0.j 0.00473726+0.j
 0.00473726+0.j 0.00473726+0.j 0.00465487+0.j 0.00469606+0.j
 0.00469606+0.j 0.00469606+0.j]

 Entry 3
Category : Ragdoll
Filename : 9r7nfja3e7t21.jpg
 Quantum State (Size: 65536):
[0.00140308+0.j 0.00096211+0.j 0.00140308+0.j 0.00112246+0.j
 0.00052114+0.j 0.00116255+0.j 0.00068149+0.j 0.00096211+0.j
 0.00060132+0.j 0.00104228+0.j]

 Entry 4
Category : Domestic_Shorthair
Filename : c8706a9f05a2f298bac780a407d472ef.jpg
 Quantum State (Size: 65536):
[0.00264252+0.j 0.00270327+0.j 0.00258177+0.j 0.00273364+0.j
 0.00276402+0.j 0.00273364+0.j 0.00273364+0.j 0.0026729 +0.j
 0